# Hospital Capacity Simulator

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import datetime

from numpy import random
from ipywidgets import *
from IPython.display import display
from datetime import datetime, timedelta, date


### Admission Data

- Generate and plot random data for planned and unplanned admissions

In [2]:
#### Widgets #### 

num_beds = IntSlider(value=275, min=100, max=500, description='Beds',  continuous_update=False)
unplan_adm_rate = FloatSlider(value=0.3, min=0.0, max=1.0, step=0.05, description='adm rate (%)', continuous_update=False)

pl_los_avg = FloatSlider(value=36.0, min=0.0, max=120.0, step=0.5,  description='avg', continuous_update=False)
pl_los_std = FloatSlider(value=12.0, min=0.0, max=60.0, step=0.1,  description='std', continuous_update=False)

unpl_los_avg = FloatSlider(value=48.0, min=0.0, max=120.0, step=0.5,  description='avg', continuous_update=False)
unpl_los_std = FloatSlider(value=16.0, min=0.0, max=60.0, step=0.1,  description='std', continuous_update=False)

pl_adm_bias_avg =   FloatSlider(value=0.0, min=-10.0, max=10.0, step=0.5,  description='avg', continuous_update=False)
pl_adm_bias_std =   FloatSlider(value=0.0, min=-10.0, max=10.0, step=0.1,  description='std', continuous_update=False)

unpl_adm_bias_avg = FloatSlider(value=1.0, min=-10.0, max=10.0, step=0.5,  description='avg', continuous_update=False)
unpl_adm_bias_std = FloatSlider(value=0.0, min=-10.0, max=10.0, step=0.1,  description='std', continuous_update=False)

today = date.today()

start = DatePicker( description='Begin:', disabled=False)
start.value = today-timedelta(days=14)

end = DatePicker(description='End:', disabled=False)
end.value = today

In [ ]:
#### Arrival Stats #### 
plan_hrly_means = np.array([[0.2, 0.2, 0.2, 0.2, 0.2, 3.5, 9.0, 13.0, 12.0, 12.0, 11.0, 11, 11, 11, 11, 10, 9, 6, 3, 1, 
                             0.2, 0.2, 0.2, 0.2]]).T

plan_hrly_std = np.array([[2.5, 2.1, 1.7, 1.6, 1.4, 1.35, 1.2, 3.5, 4.9, 5.7, 5.5, 4.4, 4.6, 4.8, 4.7, 4.6, 4.3, 4.2, 4.3, 
                            5, 5.5, 4.2, 3.3, 2.9]]).T

unplan_hrly_means = np.array([2.5, 2.1 ,1.7, 1.6, 1.4, 1.35, 1.2, 3.5, 4.9, 5.7, 5.5, 4.4, 4.6, 4.8, 4.7, 4.6, 4.3, 4.2, 4.3, 5,
                         5.5, 4.2, 3.3, 2.9])

unplan_hrly_std = np.array([0.05, 0.05, 0.05, 0.05, 0.05, 0.875, 1.5, 3, 2.75, 2.5, 3.25, 3.25, 3, 2.75, 3, 1.75, 2, 1, 0.75, 0.25, 
                        0.05, 0.05, 0.05, 0.05])

In [ ]:
#### Generators and Plots #### 

def gen_day_normal(hrly_means, hrly_std, bias, rate=None):
    if not rate: rate=1.0
    patients = [int(random.normal(loc=m.item(), scale=s.item())*rate) for m,s in zip(hrly_means + bias, hrly_std)]
    return [p if p>=0 else 0 for p in patients]


def gen_hrly_los(patients, avg, std):
    return [[int(random.normal(loc=avg, scale=std)) for x in range(a)] for a in patients]

In [ ]:
def count_days(start, end):
    if start.value and end.value:
        delta = end.value - start.value
        return delta.days

In [ ]:
def get_random_admission(start_date, end_date):
    num_days = count_days(start_date, end_date)
    string_dates = [(start_date.value + timedelta(days=i)).isoformat() for i in range(0,num_days)]
    tot_planned, tot_unplanned, hrly_pl, hrly_upl = gen_random_admissions(num_days)
    return string_dates, tot_planned, tot_unplanned, hrly_pl, hrly_upl

In [ ]:
def gen_random_admissions(num_days):
    tot_planned, tot_unplanned = [],[]
    hrly_pln, hrly_unpln = [],[]
    
    for day in range(num_days):
        rand_plan = gen_day_normal(plan_hrly_means, 
                                   plan_hrly_std, 
                                   pl_adm_bias_avg.value)
        
        rand_unplan = gen_day_normal(unplan_hrly_means, 
                                     unplan_hrly_std, 
                                     unpl_adm_bias_avg.value, 
                                     unplan_adm_rate.value)


        plan_los = gen_hrly_los(rand_plan, pl_los_avg.value, pl_los_std.value)
        unplan_los = gen_hrly_los(rand_unplan, unpl_los_avg.value, unpl_los_std.value)
        
        hrly_pln.append(plan_los)
        hrly_unpln.append(unplan_los)
        
        tot_unplanned.append(sum(rand_unplan))
        tot_planned.append(sum(rand_plan))
    
    return tot_planned, tot_unplanned, hrly_pln, hrly_unpln

In [ ]:
#### LOS ####
def los_df(hrly_los, string_dates):
    
    df = pd.DataFrame(columns=["admit","discharge", "LOS"])
    
    for day in range(len(hrly_los)):
        for hr in range(len(hrly_los[day])):
            for los in hrly_los[day][hr]:
                s = string_dates[day]+ f' {hr}:00:00' 
                admit = datetime.strptime(s, "%Y-%m-%d %H:%M:%S")
                discharge = admit + timedelta(hours=los)
                row = {"admit":admit ,"discharge":discharge, "LOS":los}
                df = df.append(row, ignore_index=True)
        
    return df

def bed_capacity(start, end, df):
    num_days = count_days(start, end)
    timestrings = []
    occupied = []
    
    for day in range(num_days):
        current = start.value + timedelta(days=day)
        for hr in range(24): 
            hour = pd.Timestamp(current.strftime("%Y-%m-%d") + f' {hr}:00:00')
            discharges = len(df[(df['discharge']<=hour)])
            admits = len(df[(df['admit']<=hour)])
            timestrings.append(hour.strftime("%m-%d-%Y %H:%M:%S"))
            occupied.append(admits - discharges)
    
    return timestrings, occupied

In [ ]:
string_dates, tot_pl, tot_unpl, hrly_pl, hrly_upl = get_random_admission(start, end)
pl_df = los_df(hrly_pl, string_dates)
times, occupied = bed_capacity(start, end, pl_df)

In [ ]:
len(tot_pl), len(tot_unpl)

### Plots

In [ ]:
def bed_capacity_matix(start, end, df):
    
    matrix_df = pd.Dataframe()
    
    num_days = count_days(start, end)
    timestrings = []
    occupied = []
    
    # for each day in date range
    for day in range(num_days):
        current_day = start.value + timedelta(days=day)
        
        # for each hour in 24 hour day
        for hr in range(24): 
            current_hour = pd.Timestamp(current_day.strftime("%Y-%m-%d") + f' {hr}:00:00')
            
            discharges = len(df[(df['discharge']<=current_hour)])
            admits = len(df[(df['admit']<=current_hour)])
            
#             timestrings.append(hour.strftime("%m-%d-%Y %H:%M:%S"))
#             occupied.append(admits - discharges)
            
    return matrix_df

In [ ]:
pl_df

In [ ]:
def autolabel(ax, rects, xpos='center'):
    ha = {'center': 'center', 'right': 'left', 'left': 'right'}
    offset = {'center': 0, 'right': 1, 'left': -1}

    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(offset[xpos]*3, 3),  # use 3 points offset
                    textcoords="offset points",  # in both directions
                    ha=ha[xpos], va='bottom')

In [ ]:
def plot_one_day():
    fig, ax = plt.subplots(1,2, figsize=(15,5))

    sns.barplot(x=np.arange(0,24,1), y=gen_day_normal(plan_hrly_means, 
                                                      plan_hrly_std, 
                                                      pl_adm_bias_avg.value), ax=ax[0]);
    ax[0].set(xlabel='Hour of the Day', ylabel='# of Patients')
    ax[0].set_title("Random Planned Admissions per Hour")

    sns.barplot(x=np.arange(0,24,1), y=gen_day_normal(unplan_hrly_means, 
                                              unplan_hrly_std, 
                                              unpl_adm_bias_avg.value , 
                                              unplan_adm_rate.value), ax=ax[1])

    ax[1].set(xlabel='Hour of the Day', ylabel='# of Patients')
    ax[1].set_title("Random Unplanned Admissions per Hour")
    plt.show();

In [ ]:
def plot_admissions(string_dates, tot_planned, tot_unplanned):
    
    ind = np.arange(len(tot_planned))  # the x locations for the groups
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots(figsize=(12, 7))
    rects1 = ax.bar(ind - width/2, tot_planned, width, label='Planned')
    rects2 = ax.bar(ind + width/2, tot_unplanned, width, label='Unplanned')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('# of patients')
    ax.set_title('Admissions Per Day')
    ax.set_xticks(ind)
    ax.set_xticklabels(string_dates)
    fig.autofmt_xdate()
    ax.legend()

    autolabel(ax, rects1, "left")
    autolabel(ax, rects2, "right")
    
    plt.show()

In [ ]:
def plot_beds(times, occupied):

    occupied = np.array(occupied)
    
    fig, ax = plt.subplots(1, figsize=(15,6))
    ax.plot(times, occupied)
    
    max_beds = float(num_beds.value)
    plt.axhline(y=max_beds, color='r', linestyle='-')
    
    ax.set_ylabel('# of patients')
    ax.legend(['Total Beds Occupied', "Max Bed Capacity"])
    
    ax.fill_between(times, occupied, max_beds, where= occupied >= max_beds, alpha=0.50,  facecolor='r', edgecolor='r', interpolate=True)
    
    plt.xticks(np.arange(0, len(times), 12.0), rotation=45)
    plt.show()

In [ ]:
#### Buttons #### 
gen_button = Button(description="Refresh")
beds_button = Button(description="Bed Capacity Calc")    

output = Output()
output2 = Output()
output3 = Output()

output.layout.height = '450px'
output2.layout.height = '450px'
output3.layout.height = '450px'

In [ ]:
def on_change(b):

    output.clear_output() 
    with output:
        plot_one_day()
    
    output2.clear_output()
    with output2:
        string_dates, tot_pl, tot_unpl, hrly_pl, hrly_upl = get_random_admission(start, end)
        plot_admissions(string_dates, tot_pl, tot_unpl) 
        
    output3.clear_output()
    with output3:
        pl_df = los_df(hrly_pl, string_dates)
        times, occupied = bed_capacity(start, end, pl_df)
        plot_beds(times, occupied)
         

In [ ]:
gen_button.on_click(on_change)
beds_button.on_click(on_change)

start.observe(on_change)
end.observe(on_change)

unplan_adm_rate.observe(on_change, names=['value'])

pl_adm_bias_avg.observe(on_change, names=['value'])
pl_adm_bias_std.observe(on_change, names=['value'])

unpl_adm_bias_avg.observe(on_change, names=['value'])
unpl_adm_bias_std.observe(on_change, names=['value'])

pl_los_avg.observe(on_change, names=['value'])
pl_los_std.observe(on_change, names=['value'])

unpl_los_avg.observe(on_change, names=['value'])
unpl_los_std.observe(on_change, names=['value'])

num_beds.observe(on_change, names=['value'])

In [ ]:
#### Forms #### 

stats_items = [HTML(value="<b>Planned Admissions:</b>"), pl_adm_bias_avg ,pl_adm_bias_std, 
               HTML(value="<b>Unplanned Admissions:</b>"), unpl_adm_bias_avg , unpl_adm_bias_std,
               unplan_adm_rate,
               gen_button]
               

stats_sidebar = Box(stats_items, layout=Layout(display='flex-grow',
                                               flex_flow='column',
                                               align_items='center',
                                               justify_content='center'))

In [ ]:
dates = Box([HBox([start, end])], layout=Layout(display='flex', flex_flow='row',
                                                            height="auto", align_items='center', 
                                                            justify_content='center', width='auto'))

gen_adms = Box([gen_button], layout=Layout(display='flex', flex_flow='row',
                                             height="auto", align_items='center', 
                                             justify_content='center', width='auto'))

In [ ]:
tab_contents = ['Sample Days', 'Total Admissions']
children = [output, output2]
tab = Tab()
tab.children = children
tab.set_title(0, 'Sample Days')
tab.set_title(1, 'Admissions Per Day')

In [ ]:
AppLayout(header = dates,
          left_sidebar=stats_sidebar,
          center=tab)

### Bed Capacity Calculator

In [ ]:
sidebar2_items = [HTML(value="<b>Total Beds:</b>"), num_beds,
                  HTML(value="<b>Planned LOS:</b>"), pl_los_avg, pl_los_std,
                  HTML(value="<b>Unplanned LOS:</b>"), unpl_los_avg, unpl_los_std]

sidebar2 = Box(sidebar2_items, layout=Layout(display='flex-grow',
                                               flex_flow='column',
                                               align_items='center',
                                               justify_content='center'))


footer2 = Box([beds_button], layout=Layout(display='flex-grow',
                                               flex_flow='row',
                                               align_items='center',
                                               justify_content='center',
                                               width="auto"))


center2 = Box([output3], layout=Layout(display='flex-grow',
                                               flex_flow='column',
                                               align_items='flex-start',
                                               width="auto"))

In [ ]:
AppLayout(left_sidebar=sidebar2,
          center=center2,
         footer=footer2)

In [ ]:
on_change(None)